### K-d tree y K-NN

Esta lista de ejercicios se basa en la implementación de k-dtree  dada [aquí](https://github.com/kapumota/CC0E5/blob/main/kd-tree.py)



#### Ejercicio 1: Implementando k-vecinos más cercanos (k-NN)

**Contexto:**
El código proporcionado tiene un método `nearestNeighbour` que encuentra el único punto más cercano. El algoritmo k-NN requiere encontrar los *k* puntos más cercanos. Esto usualmente implica mantener una lista de los *k* mejores candidatos encontrados hasta ahora, ordenados por distancia.

**Tareas:**

1.  **Diseñar `kNearestNeighbours`:**
    * En la clase `_Node`, diseña un nuevo método `kNearestNeighbours(self, target_point, k, current_k_nn=None, max_dist_in_k_nn=float('inf'))`.
    * `current_k_nn` sería una lista de tuplas (distancia, punto), mantenida ordenada por distancia, y de tamaño máximo `k`.
    * `max_dist_in_k_nn` sería la distancia al punto más lejano en `current_k_nn`.
2.  **Implementar `kNearestNeighbours`:**
    * Adapta la lógica de `nearestNeighbour`.
    * Cuando se verifica un punto en un nodo, si su distancia a `target_point` es menor que `max_dist_in_k_nn` (o si `current_k_nn` tiene menos de `k` puntos), agrégalo a `current_k_nn`, reordena, y si `current_k_nn` ahora excede `k` ítems, elimina el más lejano. Actualiza `max_dist_in_k_nn`.
    * La condición de poda para verificar la "rama más lejana" (`further_branch`) ahora usará `max_dist_in_k_nn`: si `abs(target_coord - node_coord) < max_dist_in_k_nn`.
3.  **Exponer en la clase `Kdtree`:**
    * Agrega un método público `kNearestNeighbours(self, point, k)` a la clase `Kdtree` que iniciBudincita el proceso y llame al método recursivo en la raíz.
4.  **Pruebas:**
    * Agrega casos de prueba para verificar tu implementación de `kNearestNeighbours` usando los datos de ejemplo `points_2d`. Por ejemplo, encuentra los 3 vecinos más cercanos a `Point([5, 5])`.

**Puntos de discusión:**
* ¿Cómo afecta el mantenimiento de una lista de `k` candidatos a la complejidad de la búsqueda en comparación con 1-NN?
* ¿Qué estructura de datos es adecuada para `current_k_nn` para gestionar eficientemente los `k` vecinos principales (por ejemplo, una cola de prioridad mínima de tamaño k, o una lista ordenada)? La sugerencia actual utiliza una lista ordenada.
* Compara la complejidad temporal del k-NN por fuerza bruta ($O(N \cdot D)$ para cálculos de distancia más $O(N \log k)$ o $O(N \cdot k)$ para la selección, donde N es el número de puntos, D es la dimensionalidad) con el k-NN del k-d tree (promedio $O(k \log N)$, peor caso $O(kN)$).


#### Ejercicio 2: Simulación de Kapumota y Budincita - comprendiendo la poda (Pruning) 

**Contexto:**
El texto describe un escenario donde los datos se dividen entre Kapumota y Budincita. Si Kapumota encuentra 7 vecinos y su séptimo vecino está a una distancia `d_A7`, y Kapumota puede proporcionar rápidamente una cota inferior `d_B_lower` para *cualquier* punto en su conjunto de datos (o incluso si solo su primer vecino más cercano está a `d_B1`), entonces si `d_A7 < d_B_lower` (o `d_A7 < d_B1`), no es necesario realizar una búsqueda completa en el conjunto de datos de Kapumota. Los k-d trees hacen esto implícitamente al verificar la distancia al hiperrectángulo delimitador (bounding box) de una rama.

**Tareas:**

1.  **Análisis conceptual:**
    * Considera el método `nearestNeighbour` en `_Node`. La línea `if abs(target_coord - node_coord) < current_nn_dist:` es crucial.
    * Explica cómo `abs(target_coord - node_coord)` actúa como una cota inferior de la distancia a cualquier punto en la "further_branch". Si esta cota inferior ya es mayor que `current_nn_dist`, esa rama entera se poda.
2.  **Simulación de escenario (código simple):**
    * Imagina que `tree_A` y `tree_B` son dos instancias separadas de `Kdtree`.
    * Quieres encontrar el 1-NN para un `query_point` en ambos árboles.
    * Primero, encuentra `nn_A = tree_A.nearestNeighbour(query_point)` y sea su distancia `dist_A`.
    * ¿Cómo podrías obtener una "cota inferior" para `tree_B` sin buscarlo completamente? (Pista: La clase `Cube` que representa toda la región de `tree_B` o la partición de su nodo raíz podría usarse. La distancia desde `query_point` hasta el *borde más cercano* del hiperrectángulo delimitador de `tree_B` sería una cota inferior.)
    * Si esta cota inferior es mayor que `dist_A`, no necesitas buscar en `tree_B`.
3.  **Bosquejo de código (opcional):**
    * Bosqueja cómo podrías modificar la clase `Cube` proporcionada para calcular `min_distance_to_point(self, point)` (distancia más corta desde el punto a cualquier parte del cubo).
    * ¿Cómo se relacionaría esta `min_distance_to_point` para el hiperrectángulo delimitador efectivo de la `further_branch` con `abs(target_coord - node_coord)`?

**Puntos de discusión:**
* ¿Cómo la división recursiva de los k-d trees y los hiperplanos alineados con los ejes simplifican el cálculo de estas "cotas inferiores" o "distancias a las particiones"?
* El texto afirma: "Esta capacidad de ahorrar trabajo con una cota inferior sobre la distancia a un conjunto de puntos es lo que subyace a la estructura de datos del Árbol KD.". Discute esta afirmación en el contexto del código de `nearestNeighbour`.

#### Ejercicio 3: Construcción de k-d trees y búsqueda por región 


**Tareas:**

1.  **Construcción manual:**
    * Toma un pequeño subconjunto de `points_2d`, por ejemplo, `P1 = Point([2,3]), P2 = Point([5,4]), P3 = Point([4,7]), P4 = Point([8,1])`.
    * Construye manualmente un k-d tree usando estos puntos. En cada paso:
        * Determina la dimensión de división (profundidad % K).
        * Encuentra el punto mediano para esa dimensión.
        * Muestra las sub-particiones izquierda y derecha.
    * Dibuja la estructura del árbol resultante y las particiones correspondientes en un espacio 2D.
2.  **Trazado de búsqueda por región:**
    * Define un `search_cube = Cube(Point([3,0]), Point([6,5]))`.
    * Traza la ejecución de `tree_2d.pointsInRegion(search_cube)` (o usando tu árbol construido manualmente).
    * Para cada nodo visitado:
        * Identifica la `current_node_region` (el hiperrectángulo delimitador implícitamente definido por las divisiones hasta ahora).
        * Verifica `target_region.intersects(current_node_region)`.
        * Verifica `target_region.containsPoint(self._point)`.
        * Muestra cómo el algoritmo decide recursar en los hijos izquierdo/derecho y cómo se actualiza su `current_node_region`.

**Puntos de discusión:**
* ¿Cómo afecta la elección de la mediana al balance del árbol y al rendimiento de la búsqueda? La función `median` proporcionada ordena todos los puntos en cada paso; ¿cuáles son algoritmos de búsqueda de mediana más eficientes (como se insinúa en los comentarios del código)?
* Explica la poda en `pointsInRegion`: ¿cuándo puede evitar buscar subárboles enteros?


#### Ejercicio 4: Complejidad temporal y la maldición de la dimensionalidad

**Tareas:**

1.  **Análisis de complejidad (teórico):**
    * **Construcción:** ¿Cuál es la complejidad temporal de construir el k-d tree con N puntos en D dimensiones usando la búsqueda de mediana proporcionada? (Pista: La función `median` ordena, $O(N \log N)$ o $O(N)$ para esa dimensión, y esto se hace en cada nivel).
    * **`contains` / `add` / `delete` (un solo punto):** ¿Promedio y peor caso?
    * **`nearestNeighbour` / `kNearestNeighbours`:** ¿Promedio y peor caso?
    * **`pointsInRegion`:** La complejidad depende del tamaño de la consulta y del tamaño de la salida. Discutir.
2.  **Maldición de la dimensionalidad:**
    * Investiga y explica qué significa la "maldición de la dimensionalidad" en el contexto de k-NN y estructuras de datos espaciales como los k-d trees.
    * ¿Por qué los k-d trees (y estructuras similares) tienden a tener un rendimiento pobre (acercándose a la fuerza bruta) en dimensiones muy altas? (Pista: considera cuántos hiperrectángulos delimitadores de los hijos probablemente se superpondrán con la hiperesfera/hiperrectángulo de la consulta).

**Puntos de discusión:**
* ¿Para qué rango de dimensiones son típicamente más efectivos los k-d trees?
* ¿Cuáles son las alternativas a los k-d trees para k-NN en espacios de dimensiones muy altas (por ejemplo, hashing sensible a la localidad, algoritmos de vecinos más cercanos aproximados, o incluso ball trees en algunos casos)?


#### Ejercicio 5: Ball trees y búsqueda A*


**Contexto:**
* **Ball trees:** Particionan los datos en hiperesferas anidadas en lugar de hiperrectángulos.
* **Búsqueda A*:** Un algoritmo de búsqueda de caminos que utiliza una heurística ($h(n)$) para estimar el costo hasta el objetivo y el costo real hasta el momento ($g(n)$). Explora primero los caminos con menor $f(n) = g(n) + h(n)$.

**Tareas:**

1.  **Ball trees (conceptual):**
    * Investiga brevemente "construcción de Ball trees" y "búsqueda en Ball trees".
    * ¿En qué se diferencia la partición con hiperesferas de los hiperplanos alineados con los ejes de los k-d trees?
    * ¿Cuándo podría ser ventajoso un Ball tree sobre un k-d tree? (Considera la distribución de los datos, tipos de consultas).
    * ¿Cómo funcionaría la condición de poda en un Ball trees para la búsqueda del vecino más cercano? (Pista: distancia desde el punto de consulta hasta la superficie de una bola, y el radio de la bola).
2.  **Analogía con la búsqueda A*:**
    * En el contexto de `_Node.nearestNeighbour`:
        * ¿Qué podría considerarse el "estado" o "nodo" en el sentido de la búsqueda A*? (Un nodo del k-d tree).
        * ¿Qué es $g(n)$, el costo desde el inicio hasta el estado/nodo actual? (Esto es menos directo, pero considera el camino tomado en el árbol).
        * ¿Qué actúa como $h(n)$, la estimación heurística del costo desde el estado/nodo actual hasta el objetivo (el verdadero vecino más cercano)? (Pista: La distancia desde el punto de consulta hasta el *hiperrectángulo delimitador* de la región de un nodo del k-d tree).
        * ¿Cómo prioriza el algoritmo qué nodos del k-d tree (ramas) explorar, y cómo se relaciona esto con $f(n)$ de A*? (El algoritmo explora primero la rama "más cercana", y solo explora la rama "más lejana" si la distancia optimista a ella es menor que la mejor encontrada hasta ahora).

**Puntos de discusión:**
* ¿Por qué la distancia al hiperrectángulo delimitador de un subárbol es una heurística *admisible* para A* (es decir, nunca sobrestima la verdadera distancia al mejor punto dentro de ese subárbol)?
* ¿Garantiza la búsqueda en el k-d tree encontrar la solución óptima (el verdadero vecino más cercano)? ¿Por qué?


#### Ejercicio 6: Conexiones con el aprendizaje automático y exploración adicional

**Tareas:**

1.  **K-NN como clasificador/regresor:**
    * Explica cómo se puede usar k-NN para tareas de clasificación (voto mayoritario entre k vecinos) y tareas de regresión (promedio de los valores de k vecinos).
    * ¿Cuáles son las principales ventajas de k-NN como algoritmo de aprendizaje automático? (por ejemplo, simplicidad, sin fase de entrenamiento explícita, a menudo llamado "aprendiz perezoso" o "lazy learner").
    * ¿Cuáles son sus principales desventajas? (por ejemplo, costo computacional en el momento de la consulta, sensibilidad al escalado de características, maldición de la dimensionalidad).
2.  **Impacto de `K` en k-NN:**
    * ¿Cómo afecta la elección de `k` al sesgo y la varianza del modelo k-NN?
    * ¿Qué sucede si `k` es demasiado pequeño? ¿Demasiado grande?
3.  **Escalado de características:**
    * ¿Por qué el escalado de características (por ejemplo, normalización o estandarización) suele ser crucial antes de aplicar k-NN (y, por lo tanto, al usar un k-d tree)? ¿Cómo afectarían las características no escaladas a los cálculos de distancia y al comportamiento de las divisiones del k-d tree?
4.  **Mejorando el k-d tree:**
    * El método `delete` proporcionado está marcado como "simplificado". Investiga y discute las complejidades de la eliminación robusta en k-d trees y cómo puede llevar a un desequilibrio del árbol. ¿Qué estrategias existen para manejar esto (por ejemplo, reconstrucción periódica, marcar nodos como eliminados)?
    * La búsqueda de la mediana ordena todos los puntos en el subconjunto actual para una dimensión dada. ¿Cómo mejoraría la complejidad de la construcción el uso de un verdadero algoritmo de mediana de medianas en tiempo lineal?

**Puntos de discusión:**
* ¿Cuándo elegirías un k-NN optimizado con k-d tree sobre otros algoritmos como SVM,  árboles de decisión o redes neuronales?
* Considera aplicaciones del mundo real donde el k-NN acelerado por k-d trees podría ser particularmente útil.

#### Ejercicio 7: Múltiples métricas de distancia en `Point`

**Contexto:**
La clase `Point` actual implementa la distancia Euclidiana. Sin embargo, en diferentes aplicaciones, otras métricas de distancia como la distancia de Manhattan (o "city block") pueden ser más apropiadas.

**Tareas:**

1.  **Implementar distancia de Manhattan:**
    * Añade un nuevo método `distanceToManhattan(self, other_point)` a la clase `Point`.
    * Esta distancia se calcula como la suma de las diferencias absolutas de sus coordenadas:
        $d_M(p, q) = \sum_{i=1}^{D} |p_i - q_i|$
    * Asegúrate de que maneje las mismas validaciones que `distanceTo` (misma dimensionalidad).
2.  **Modificar `Kdtree` para usar diferentes métricas:**
    * Considera cómo podrías modificar la clase `Kdtree` (y `_Node`) para permitir que el usuario especifique qué métrica de distancia usar al construir el árbol o al realizar búsquedas. Esto podría implicar pasar una función de distancia como parámetro.
    * *Nota:* La lógica de poda del k-d tree está intrínsecamente ligada a la distancia Euclidiana (o métricas $L_p$ donde las proyecciones sobre los ejes son relevantes para las cotas). Cambiar la métrica podría requerir repensar la validez de la poda actual si la nueva métrica no se comporta bien con las divisiones axiales. Para este ejercicio, centrarse en la clase `Point` es suficiente, pero la discusión sobre el impacto en el `Kdtree` es valiosa.
3.  **Pruebas:**
    * Crea instancias de `Point` y prueba tu nuevo método `distanceToManhattan`. Compara los resultados con `distanceTo` (Euclidiana) para los mismos puntos.

**Puntos de discusión:**
* ¿En qué tipo de escenarios o con qué tipo de datos podría ser preferible la distancia de Manhattan sobre la Euclidiana?
* Si se usara la distancia de Manhattan en el algoritmo k-NN, ¿cómo cambiarían las "vecindades"?
* ¿Sigue siendo válida la estrategia de poda del k-d tree (que se basa en la distancia perpendicular a los hiperplanos de división) si la métrica principal de "cercanía" es la de Manhattan? ¿Por qué sí o por qué no?

#### Ejercicio 8: Puntos en un nivel específico del árbol

**Contexto:**
A veces es útil analizar la distribución de los datos o la estructura del árbol por niveles. Por ejemplo, para entender cómo se han dividido los datos en las primeras etapas de la construcción del árbol.

**Tareas:**

1.  **Implementar `_points_at_depth_recursive` en `_Node`:**
    * Añade un método auxiliar recursivo `_points_at_depth_recursive(self, target_depth, current_depth)` a la clase `_Node`.
    * Si `current_depth == target_depth`, el método debe producir (yield) el punto del nodo actual (`self._point`).
    * Si `current_depth < target_depth`, el método debe llamar recursivamente a sí mismo en los hijos izquierdo y derecho (si existen), incrementando `current_depth`.
2.  **Implementar `points_at_depth` en `Kdtree`:**
    * Añade un método público `points_at_depth(self, depth)` a la clase `Kdtree`.
    * Este método debe manejar el caso de un árbol vacío o si la `depth` solicitada es negativa o mayor que la altura del árbol.
    * Debe llamar al método recursivo en el nodo raíz, iniciando `current_depth` en 0.
    * Debe devolver una lista o un iterador de los puntos encontrados.
3.  **Pruebas:**
    * Utiliza el `tree_2d` de ejemplo del código.
    * Llama a `tree_2d.points_at_depth(0)`, `tree_2d.points_at_depth(1)`, `tree_2d.points_at_depth(2)`, etc., y verifica que los puntos devueltos sean los correctos según la estructura esperada del árbol.

**Puntos de discusión:**
* ¿Cuál es la relación entre la profundidad máxima para la cual este método devuelve puntos y la altura del árbol (`tree.height`)?
* ¿Cómo podrías modificar este método para devolver no solo los puntos, sino también los propios nodos o la región (hipercubo) que representa cada nodo en esa profundidad?
* ¿Para qué tipo de análisis o visualización podría ser útil obtener los puntos por nivel?

#### Ejercicio 9: Encontrar el vecino más lejano

**Contexto:**
Mientras que la búsqueda del vecino más cercano es común, encontrar el vecino más lejano también puede tener aplicaciones (por ejemplo, en detección de anomalías o para entender la dispersión de un conjunto de datos). Para un k-d tree, esto requiere una estrategia de poda diferente.

**Tareas:**

1.  **Diseñar la lógica de búsqueda del vecino más lejano:**
    * Se necesitará un método recursivo en `_Node`, similar a `nearestNeighbour`, pero que lleve un seguimiento del punto más lejano encontrado hasta ahora y su distancia.
    * Llamémoslo `_farthest_neighbour_recursive(self, target_point, current_farthest_point_info, current_node_region)`.
    * `current_farthest_point_info` podría ser una tupla `(distancia_max, punto_mas_lejano)`.
2.  **Implementar `_farthest_neighbour_recursive` en `_Node`:**
    * **Visitar nodo actual:** Calcula la distancia desde `target_point` al punto del nodo actual (`self._point`). Si es mayor que la `distancia_max` actual, actualiza `current_farthest_point_info`.
    * **Estrategia de ramificación:** Decide el orden para visitar los hijos. Podrías, por ejemplo, priorizar la rama cuyo hiperrectángulo delimitador tenga un vértice que esté más lejos del `target_point`.
    * **Poda (Pruning):** Esta es la parte clave. Para podar una rama (por ejemplo, `other_branch_node` con su región `other_branch_region`):
        * Calcula la distancia desde `target_point` al **vértice más lejano** del `other_branch_region`. Esta es la máxima distancia posible que un punto dentro de esa región podría tener al `target_point`.
        * Si esta "máxima distancia posible en la rama" es *menor* que la `distancia_max` actual (la distancia al vecino más lejano encontrado *hasta ahora*), entonces no hay necesidad de explorar esa rama.
        * *Nota:* La clase `Cube` necesitará un método para calcular la distancia desde un punto externo a su vértice más lejano, o alternativamente, la máxima distancia al cuadrado para evitar `sqrt`.
3.  **Implementar `farthestNeighbour` en `Kdtree`:**
    * Añade un método público `farthestNeighbour(self, target_point)` a `Kdtree`.
    * Inicializa `current_farthest_point_info` con el primer punto que encuentres y su distancia, o maneja el caso de árbol vacío.
    * Llama al método recursivo en la raíz.
4.  **Pruebas:**
    * Con `tree_2d`, encuentra el vecino más lejano a `Point([0,0])` o a un punto en el centro del conjunto de datos. Verifica manualmente si el resultado es correcto.

**Puntos de discusión:**
* Compara la complejidad y la efectividad de la poda para la búsqueda del vecino más lejano versus la del vecino más cercano en un k-d tree.
* ¿Cómo se implementaría el cálculo de la "distancia al vértice más lejano de un hipercubo (`Cube`)"?
* ¿Existen configuraciones de puntos o puntos de consulta donde la poda para el vecino más lejano sea poco efectiva?



In [ ]:
### Tus respuestas

### Proyecto: Exploración avanzada de búsqueda de vecinos cercanos con K-d trees y alternativas

#### Módulo 1: Optimización y análisis del K-d tree 

#### Sesión 1.1: Del vecino más cercano a k-vecinos (k-NN) y primeras mediciones

* **Introducción:**
    * Presentación del problema: "Un cliente con un K-d tree funcional ahora requiere encontrar los *k* vecinos más cercanos, no solo uno."
    * Discusión inicial sobre la necesidad de eficiencia.
* **Actividad principal (codificación):**
    1.  **Implementación de `kNearestNeighbours`:**
        * Modificar el método `nearestNeighbour` existente para convertirlo en `kNearestNeighbours`.
        * **Requisito de eficiencia:** Utilizar un **max-heap de tamaño fijo `k`** para mantener el ranking provisional de los `k` mejores candidatos encontrados hasta ahora, evitando ordenaciones completas en cada paso.
    2.  **Instrumentación básica del código:**
        * Añadir contadores para:
            * Número de nodos visitados.
            * Número de ramas podadas (debido a la distancia al hiperplano de corte).
        * Utilizar `time.perf_counter()` para medir el tiempo de ejecución de las búsquedas.
* **Experimentación y análisis inicial:**
    1.  Generar conjuntos de datos (nubes de puntos aleatorios) con:
        * Dimensionalidades: 2, 5, y 10.
        * Tamaños (N): Creciente desde 1,000 hasta 100,000 puntos.
    2.  Comparar el rendimiento de la búsqueda k-NN con el K-d tree vs. un algoritmo de fuerza bruta.
    3.  Visualizar en una gráfica el tiempo de ejecución (K-d tree vs. Fuerza Bruta) en función de la dimensionalidad. Observar el "cruce" de las curvas.
* **Discusión en clase:**
    * La "maldición de la dimensionalidad": ¿Qué significa y cómo se manifiesta en los resultados observados?
* **Tarea 1:**
    1.  Finalizar la instrumentación del código para `kNearestNeighbours`.
    2.  Generar y entregar dos gráficos (archivos PNG):
        * Gráfico 1: Tiempo medio de búsqueda vs. tamaño del dataset (N) para una D fija.
        * Gráfico 2: Nodos visitados vs. tamaño del dataset (N) para una D fija.
        (Ambos gráficos deben mostrar K-d tree y fuerza bruta).


#### Sesión 1.2: Profundizando en la teoría y optimizando con cajas delimitadoras (bounding boxes)

* **Revisión y discusión:**
    * Analizar las gráficas de la tarea. Pregunta guía: "¿Por qué las curvas de rendimiento (especialmente nodos visitados) tienen esa forma particular?"
* **Componente teórico:**
    * Cada estudiante investiga y prepara una breve explicación (máx. 3 ideas clave/diapositivas) sobre:
        * Complejidad de construcción del K-d tree: $O(N \log N)$ en promedio.
        * Complejidad de búsqueda promedio: $O(\log N)$ para 1-NN (o $O(k + \log N)$ / $O(k \log N)$ para k-NN).
        * Peor caso de rendimiento: ¿Cuándo y por qué el rendimiento se degrada (ej. puntos colineales o estructura de datos degenerada)?
* **Actividad principal (codificación y optimización):**
    1.  **Integración de cajas delimitadoras (bounding boxes):**
        * Modificar la estructura de cada nodo del K-d tree para almacenar explícitamente sus `min_bounds` y `max_bounds` (el hiperrectángulo mínimo que encierra todos los puntos en el subárbol rooted en ese nodo).
        * Estos bounds se calculan durante la construcción del árbol.
    2.  **Optimización de la poda:**
        * Modificar la lógica de búsqueda (`kNearestNeighbours`) para utilizar estas cajas delimitadoras.
        * Calcular la distancia desde el punto de consulta hasta la caja delimitadora de una rama.
        * Si esta distancia (cota inferior) ya es mayor que la distancia al k-ésimo vecino más lejano encontrado hasta ahora, podar la rama completa sin necesidad de visitarla.
* **Experimentación y tarea:**
    1.  Ejecutar un script que repita los experimentos de la Sesión 1.1 (tiempo y nodos visitados), pero ahora comparando:
        * K-d tree con poda original (solo hiperplano).
        * K-d tree con poda mejorada usando bounding boxes.
    2.  Generar y entregar dos nuevos gráficos (png) que muestren el impacto de la poda con bounding boxes, especialmente en el número de nodos visitados.

#### Módulo 2: Escenarios distribuidos y estructuras de datos alternativas

#### Sesión 2.1: K-d trees en un entorno distribuido

* **Introducción del escenario:**
    * "Kapumota y Budincita gestionan cada una la mitad de un mega-dataset en servidores separados. Un cliente necesita realizar búsquedas k-NN sobre el dataset completo."
    * Objetivo: Minimizar el cómputo caro y las transferencias de datos.
* **Actividad principal (codificación y simulación):**
    1.  **Simulación de sistema distribuido:**
        * Utilizar el módulo `multiprocessing` de Python.
        * Crear dos procesos "servidor" (Kapumota, Budincita), cada uno con una instancia de `Kdtree` sobre una porción del dataset.
        * Crear un proceso "cliente".
        * Utilizar `multiprocessing.Queue` para la comunicación (mensajes) entre cliente y servidores.
    2.  **Implementación del protocolo de consulta eficiente:**
        * **Paso 1 (Cliente):** Solicitar a *ambos* servidores una "cota inferior" de distancia para un punto de consulta dado. Esta cota puede ser la distancia al vecino más cercano en su respectivo dataset, o una estimación más rápida (ej. distancia a la bounding box de la raíz de su árbol).
        * **Paso 2 (Cliente):** Elegir el servidor "más prometedor" (con la menor cota inferior). Solicitarle sus `k` vecinos más cercanos.
        * **Paso 3 (Cliente):** Sea `d_k` la distancia al k-ésimo vecino retornado por el primer servidor. Si `d_k` es menor que la cota inferior del *segundo* servidor, la búsqueda termina. ¡Se ahorra la consulta completa al segundo servidor!
        * Si no, se realiza la consulta completa al segundo servidor y se combinan los resultados.
* **Experimentación y análisis:**
    * Medir:
        * Número promedio de llamadas de red "caras" (búsqueda k-NN completa) ahorradas.
        * Speed-up conseguido en comparación con una consulta secuencial a ambos servidores o una consulta ingenua que siempre pide los k vecinos a ambos.
* **Tarea 2:**
    1.  Generalizar la simulación para `M` servidores.
    2.  Implementar y comparar dos estrategias para la obtención de cotas inferiores de los `M` servidores:
        * **Secuencial:** Consultar las cotas una por una.
        * **Paralela:** Lanzar todas las solicitudes de cotas en paralelo (usando `multiprocessing` o `threading` para las esperas).
    3.  Justificar la elección de la mejor estrategia y presentar resultados de speed-up para `M=2, 4, 8`.


#### Sesión 2.2: Introducción a los Ball-trees 

* **Introducción a los Ball-trees:**
    * Presentar el Ball-tree como una alternativa al K-d tree, que particiona el espacio usando hiperesferas en lugar de hiperplanos.
    * Discutir las diferencias conceptuales en la construcción y la búsqueda.
* **Actividad principal (codificación y comparación):**
    1.  **Implementación de un Ball-tree (simplificado):**
        * **Estructura del nodo:** Cada nodo representa una hiperesfera (centro y radio) que encapsula todos los puntos en su subárbol.
        * **Construción (simplificada):** En cada nodo, elegir un punto pivote, y asignar puntos a dos nuevas sub-esferas hijas basándose en su distancia al pivote o alguna otra heurística de partición.
        * **Cálculo de cota inferior para poda:** La distancia mínima desde un punto de consulta `q` a cualquier punto dentro de una bola (centro `c`, radio `r`) es $| \text{distancia}(q,c) - r |$. Esto se usa para la poda de manera análoga a los K-d trees.
    2.  **Reutilización de benchmarks:**
        * Reutilizar el framework de benchmarking existente.
        * Comparar el rendimiento (tiempo de búsqueda, nodos visitados) del Ball-tree implementado con el K-d tree (versión con bounding boxes).
        * Prestar especial atención a dimensiones más altas (ej. D = 5, 10, 15, 20).
* **Análisis de resultados:**
    * Generar gráficos comparativos K-d tree vs. Ball-tree en función de la dimensionalidad.
    * Observar si el Ball-tree muestra mejor resiliencia a la maldición de la dimensionalidad en el rango probado.
* **Debate final:**
    * "¿Cuándo elegiríamos un K-d tree, un Ball-tree, o un VP-tree?"
    * "¿En qué escenarios sería apropiado abandonar las estructuras de búsqueda exacta y optar por métodos aproximados como Locality-Sensitive Hashing (LSH)?"
* **Ejercicio:**
    * Investigar e implementar una versión básica de LSH para k-NN aproximado.



In [ ]:
## Tu respuesta

### Trabajo final de consolidación 

**Objetivo:** Integrar los aprendizajes, explorar extensiones y reflexionar sobre el proyecto.

**Tareas (a elegir o combinar, los equipos deben completar al menos dos de las tres primeras):**

1.  **Búsqueda aproximada ($\epsilon$-NN) en K-d tree:**
    * Modificar la condición de poda en el K-d tree: en lugar de `podar si cota_inferior_rama > distancia_k_actual`, usar `podar si cota_inferior_rama > (1 +` $\epsilon$ ` * distancia_k_actual`.
    * Experimentar con valores de $\epsilon$ (ej. 0.1, 0.3).
    * Medir el trade-off entre speed-up y precisión (ej. porcentaje de veces que se encuentra el mismo conjunto de k-vecinos que la búsqueda exacta).
2.  **Dinámica de rebalanceo del K-d tree:**
    * Implementar operaciones de `add` y `delete` (la `delete` puede ser simplificada, marcando nodos como borrados).
    * Realizar un experimento: insertar y luego borrar un gran número de puntos (ej. 1 millón de inserciones aleatorias, seguidas de 500,000 eliminaciones aleatorias).
    * Medir la altura del árbol antes y después, y el rendimiento de búsqueda.
    * Implementar una heurística de rebalanceo simple: si la altura del árbol supera `C * log N` (ej. `C = 1.5` o `2`), reconstruir el árbol completo. Mostrar el impacto de esta heurística.
3.  **Mejoras adicionales al Ball-tree o K-d tree:**
    * Investigar e implementar una estrategia de selección de pivote o eje de división más sofisticada para el K-d tree o Ball-tree.
    * Implementar una construcción de Ball-tree más robusta.
4.  **README  (Obligatorio):**
    * Un documento conciso (formato Markdown) que incluya:
        * Breve resumen de las implementaciones y experimentos realizados.
        * Principales hallazgos y sorpresas.
        * Discusión sobre qué heurísticas o estructuras funcionaron mejor y bajo qué condiciones.
        * Argumentación sobre cuándo optarían por K-d tree, Ball-tree, o LSH en un problema real.
        * Gráficos clave generados durante el proyecto.

**Entrega final:**
* Repositorio de código con todas las implementaciones.
* El README reflexivo.
* Script (ej. `Makefile` o `run_all_reports.sh`) que permita al docente generar automáticamente los principales gráficos y métricas del proyecto.


In [ ]:
## Tus respuestas